<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TensorFlow_with_GPU7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytorch with TPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
!export XLA_USE_BF16=1
!pip install transformers

TF version 2.2.0


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_PREDICT_BATCHNORM_CONVERT_SCALE_PASS_H
#define MINDSPORE_PREDICT_BATCHNORM_CONVERT_SCALE_PASS_H

#include <unordered_map>
#include <memory>
#include <string>
#include "tools/converter/legacy_optimizer/fusion/fusion_pass.h"
#include "tools/common/graph_util.h"

namespace mindspore {
namespace lite {
struct BNWeightTensors {
  TensorT *meanTensor = nullptr;
  TensorT *varianceTensor = nullptr;
  TensorT *scaleTensor = nullptr;
  TensorT *biasTensor = nullptr;
};
class BatchNormConvertScalePass : public FusionPass {
 public:
  BatchNormConvertScalePass() = default;

  ~BatchNormConvertScalePass() override;

  STATUS DefinePattern() override;

  STATUS DoFusion(MetaGraphT *graph, const std::string &patternName,
                  std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath) override;

  STATUS Run(MetaGraphT *graph) override;

 protected:
  STATUS GetTransParam(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath);

  // Get and check BNNode weight tensor
  STATUS GetBnWeightTensors(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath, BNWeightTensors &bnWeightTensors);

  STATUS GetBnEpsilon(MetaGraphT *graph, float &eps);

  STATUS FindNodes(MetaGraphT *graph, const std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath);

  STATUS GenNewScaleTensor(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath);

  STATUS ConvertBNToScale(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath);

  CNodeT *inputNode = nullptr;
  CNodeT *bnNode = nullptr;

  std::string inputOpName = "Input";
  std::string bnOpName = "BatchNorm";
  std::string bnPatternName = "BnToScaleFusion";
  int bnChannel = 0;
  TensorT *bnMeanTensor = nullptr;
  float *transScale = nullptr;
  float *transBias = nullptr;
  std::unique_ptr<TensorT> newScaleWeightTensor = nullptr;
  std::unique_ptr<TensorT> newScaleBiasTensor = nullptr;

  OpDefCopyer ScaleOpCopyer = [](CNodeT *inOpDef) -> std::unique_ptr<CNodeT> {
    std::unique_ptr<CNodeT> newOpDef(new(std::nothrow) CNodeT);
    if (newOpDef == nullptr) {
      MS_LOG(ERROR) << "new OpDefT failed";
      return nullptr;
    }
    newOpDef->name = inOpDef->name;
    newOpDef->quantType = inOpDef->quantType;
    newOpDef->primitive = std::make_unique<schema::PrimitiveT>();
    newOpDef->primitive->value.type = schema::PrimitiveType_Scale;
    auto scaleParam = new(std::nothrow) ScaleT;
    if (scaleParam == nullptr) {
      MS_LOG(ERROR) << "new scaleParam failed";
      return nullptr;
    }
    auto inParam = inOpDef->primitive->value.AsScale();
    MS_ASSERT(inParam != nullptr);
    scaleParam->axis = inParam->axis;
    newOpDef->primitive->value.value = scaleParam;
    return std::move(newOpDef);
  };
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_PREDICT_BATCHNORM_CONVERT_SCALE_PASS_H


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include "tools/converter/legacy_optimizer/fusion/batchnorm_convert_scale_pass.h"
#include <cfloat>
#include <memory>
#include <string>
#include <unordered_map>
#include <utility>
#include <vector>
#include "utils/log_adapter.h"
#include "tools/common/graph_util.h"
#include "tools/common/tensor_util.h"
#include "include/errorcode.h"
#include "schema/inner/model_generated.h"
#include "src/common/op_utils.h"

namespace mindspore {
namespace lite {
#define CAFFE_BATCHNORM_OP_WEIGHT_NUM 2
#define TF_BATCHNORM_OP_WEIGHT_NUM 4
#define CAFFE_BATCHNORM_MEAN_INDEX 0
#define CAFFE_BATCHNORM_VARIANCE_INDEX 1
#define TF_BATCHNORM_SCALE_INDEX 0
#define TF_BATCHNORM_BIAS_INDEX 1
#define TF_BATCHNORM_MEAN_INDEX 2
#define TF_BATCHNORM_VARIANCE_INDEX 3
namespace {
constexpr const float EPS = 1e-8;
constexpr const float EPS_DEFAULT_FLOAT = 1e-5;
constexpr const float POW_NUM = 0.5;
constexpr const int32_t NCHW_DIM_C = 1;
}
STATUS BatchNormConvertScalePass::Run(MetaGraphT *graph) { return FusionPass::Run(graph); }

STATUS BatchNormConvertScalePass::DefinePattern() {
  // with preNode
  {
    auto inputOp = std::make_shared<PatternOp>();
    inputOp->id = inputOpName;
    inputOp->types = {schema::PrimitiveType_NONE};
    inputOp->isPlaceHold = true;

    auto bnOp = std::make_shared<PatternOp>();
    bnOp->id = bnOpName;
    bnOp->types = {schema::PrimitiveType_FusedBatchNorm, schema::PrimitiveType_BatchNorm};
    bnOp->left = inputOp;

    std::unique_ptr<FusionPattern> fusionPattern(new(std::nothrow) FusionPattern(bnPatternName));
    if (fusionPattern == nullptr) {
      MS_LOG(ERROR) << "new fusionPattern failed";
      return RET_ERROR;
    }
    fusionPattern->AddPatternOp(inputOp);
    fusionPattern->AddPatternOp(bnOp);
    fusionPattern->Finish();

    this->patterns.emplace_back(fusionPattern.release());
  }

  return RET_OK;
}
STATUS BatchNormConvertScalePass::DoFusion(MetaGraphT *graph, const std::string &patternName,
                                           std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath) {
  MS_ASSERT(graph != nullptr);
  if (patternName != bnPatternName) {
    MS_LOG(ERROR) << "BatchNormConvertScale-Fusion match failed";
    return RET_PARAM_INVALID;
  }
  auto status = FindNodes(graph, matchedPath);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "FindNodes failed: " << status;
    return status;
  }
  auto bnPath = matchedPath.at(bnOpName);
  status = GetTransParam(graph, bnPath);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "GetTransParam failed: " << status;
    return status;
  }

  status = GenNewScaleTensor(graph, bnPath);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "GenNewScaleTensor failed: " << status;
    return status;
  }

  status = ConvertBNToScale(graph, bnPath);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "GenNewScaleTensor failed: " << status;
    return status;
  }
  return RET_OK;
}
STATUS BatchNormConvertScalePass::ConvertBNToScale(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath) {
  auto scaleNode = std::unique_ptr<CNodeT>(new(std::nothrow) CNodeT);
  if (scaleNode == nullptr) {
    MS_LOG(ERROR) << "new TransNode failed";
    return RET_ERROR;
  }
  scaleNode->name = bnNode->name;
  scaleNode->primitive = std::make_unique<schema::PrimitiveT>();
  if (scaleNode->primitive == nullptr) {
    MS_LOG(ERROR) << "op->primitive is null";
    return RET_NULL_PTR;
  }
  scaleNode->primitive->value.type = schema::PrimitiveType_Scale;
  std::unique_ptr<ScaleT> scaleParam(new ScaleT());
  if (scaleParam == nullptr) {
    MS_LOG(ERROR) << "new transposeParam failed";
    return RET_ERROR;
  }
  scaleParam->axis = NCHW_DIM_C;
  scaleNode->primitive->value.value = scaleParam.release();
  auto scaleIter = graph->nodes.begin() + bnPath->nodeIdx;
  STATUS errorCode = RET_OK;
  scaleIter =
      InsertNode(graph, scaleIter, kBefore, 0, std::move(scaleNode), &errorCode, ScaleOpCopyer, false);
  if (errorCode != RET_OK) {
    MS_LOG(ERROR) << "InsertNode failed: %d";  // errorCode);
    return errorCode;
  }
  auto &newScaleNode = *(scaleIter - 1);
  graph->allTensors.emplace_back(std::move(newScaleWeightTensor));
  auto weightTensorIdx = graph->allTensors.size() - 1;
  graph->allTensors.emplace_back(std::move(newScaleBiasTensor));
  auto biasTensorIdx = graph->allTensors.size() - 1;
  newScaleNode->inputIndex.push_back(weightTensorIdx);
  newScaleNode->inputIndex.push_back(biasTensorIdx);
  // delete bn node
  auto status = IsolateOneWayNode(graph, bnPath->nodeIdx + 1, true);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "IsolateOneWayNode " << bnNode->name.c_str() << " failed, error: " << status;
    return status;
  }
  return RET_OK;
}
STATUS BatchNormConvertScalePass::GenNewScaleTensor(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath) {
  MS_ASSERT(graph != nullptr);
  GetTransParam(graph, bnPath);
  newScaleWeightTensor = std::unique_ptr<TensorT>(new(std::nothrow) TensorT);
  if (newScaleWeightTensor == nullptr) {
    MS_LOG(ERROR) << "new weightTensor failed";
    return RET_ERROR;
  }
  newScaleWeightTensor->dataType = bnMeanTensor->dataType;
  newScaleWeightTensor->format = bnMeanTensor->format;
  newScaleWeightTensor->refCount = schema::NodeType_ValueNode;
  newScaleWeightTensor->dims = bnMeanTensor->dims;
  auto weightShapeSize = GetShapeSize(*bnMeanTensor);
  newScaleWeightTensor->data.resize(weightShapeSize * sizeof(float));
  auto ret = memcpy_s(newScaleWeightTensor->data.data(), weightShapeSize * sizeof(float), transScale,
                      weightShapeSize * sizeof(float));
  if (ret != RET_OK) {
    delete transScale;
    MS_LOG(ERROR) << "memcpy error: " << ret;
    return RET_ERROR;
  }

  newScaleBiasTensor = std::unique_ptr<TensorT>(new(std::nothrow) TensorT);
  if (newScaleBiasTensor == nullptr) {
    MS_LOG(ERROR) << "new weightTensor failed";
    return RET_ERROR;
  }
  newScaleBiasTensor->dataType = bnMeanTensor->dataType;
  newScaleBiasTensor->format = bnMeanTensor->format;

  newScaleBiasTensor->refCount = schema::NodeType_ValueNode;
  newScaleBiasTensor->dims = bnMeanTensor->dims;
  weightShapeSize = GetShapeSize(*bnMeanTensor);
  newScaleBiasTensor->data.resize(weightShapeSize * sizeof(float));
  ret = memcpy_s(newScaleBiasTensor->data.data(), weightShapeSize * sizeof(float), transBias,
                 weightShapeSize * sizeof(float));
  if (ret != RET_OK) {
    delete transBias;
    MS_LOG(ERROR) << "memcpy error: " << ret;
    return RET_ERROR;
  }
  return RET_OK;
}

STATUS BatchNormConvertScalePass::FindNodes(MetaGraphT *graph,
                                            const std::unordered_map<std::string, std::shared_ptr<Path>> &matchedPath) {
  MS_ASSERT(graph != nullptr);
  auto inputPath = matchedPath.at(inputOpName);
  auto bnPath = matchedPath.at(bnOpName);
  MS_ASSERT(inputPath != nullptr);
  MS_ASSERT(bnPath != nullptr);
  if (inputPath->subGraphIdx != bnPath->subGraphIdx) {
    MS_LOG(ERROR) << "matched nodes should from same subGraph";
    return RET_ERROR;
  }
  MS_ASSERT(graph->nodes.size() > inputPath->nodeIdx);
  MS_ASSERT(graph->nodes.size() > bnPath->nodeIdx);
  inputNode = graph->nodes.at(inputPath->nodeIdx).get();
  bnNode = graph->nodes.at(bnPath->nodeIdx).get();
  MS_ASSERT(inputNode != nullptr);
  MS_ASSERT(bnNode != nullptr);
  return RET_OK;
}
STATUS BatchNormConvertScalePass::GetTransParam(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath) {
  MS_ASSERT(graph != nullptr);
  MS_ASSERT(bnPath != nullptr);

  BNWeightTensors bnWeightTensors;

  auto status = GetBnWeightTensors(graph, bnPath, bnWeightTensors);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "GetBnWeightTensors error";
    return status;
  }
  auto *meanTensor = bnWeightTensors.meanTensor;
  auto *varianceTensor = bnWeightTensors.varianceTensor;
  auto *scaleTensor = bnWeightTensors.scaleTensor;
  auto *biasTensor = bnWeightTensors.biasTensor;

  auto *meanData = reinterpret_cast<float *>(meanTensor->data.data());
  auto *varianceData = reinterpret_cast<float *>(varianceTensor->data.data());

  float eps = EPS_DEFAULT_FLOAT;
  status = GetBnEpsilon(graph, eps);
  if (status != RET_OK) {
    MS_LOG(ERROR) << "GetBnEpsilon failed";
    return status;
  }
  this->transScale = new(std::nothrow) float[bnChannel];
  this->transBias = new(std::nothrow) float[bnChannel];
  // cal transScale, tf : scale/sqrt(variance + eps); caffe : 1/sqrt(variance + eps)
  if (memcpy_s(transScale, bnChannel * sizeof(float), varianceData, bnChannel * sizeof(float)) != 0) {
    MS_LOG(ERROR) << "memcpy_s transScale error";
    return RET_ERROR;
  }
  // 1/sqrt(variance + eps)
  for (int32_t i = 0; i < bnChannel; i++) {
    float tmp = transScale[i] + eps;
    tmp = pow(tmp, POW_NUM);
    transScale[i] = 1 / tmp;
  }

  if (scaleTensor != nullptr) {
    auto *scaleData = reinterpret_cast<float *>(scaleTensor->data.data());
    // scale/sqrt(variance + eps)
    for (int32_t i = 0; i < bnChannel; i++) {
      transScale[i] *= scaleData[i];
    }
  }

  // cal transBias, tf : -scale*mean/sqrt(variance + eps) + bias; caffe : -mean/sqrt(variance + eps)
  //-mean/sqrt(variance + eps)
  for (int32_t i = 0; i < bnChannel; i++) {
    transBias[i] = -meanData[i] * transScale[i];
  }

  if (biasTensor != nullptr) {
    auto *biasData = reinterpret_cast<float *>(biasTensor->data.data());
    //-scale*mean/sqrt(variance + eps) + bias
    for (int32_t i = 0; i < bnChannel; i++) {
      transBias[i] += biasData[i];
    }
  }

  return RET_OK;
}

// BatchNorm weight Tensor definition:
// caffe
//   estimated_mean  --0
//   estimated_variance  --1
// tensorflow
//   scale    -- 0
//   bias        --1
//   estimated_mean  --2
//   estimated_variance  --3
STATUS BatchNormConvertScalePass::GetBnWeightTensors(MetaGraphT *graph, const std::shared_ptr<Path> &bnPath,
                                                     BNWeightTensors &bnWeightTensors) {
  if (graph == nullptr || bnPath == nullptr) {
    MS_LOG(ERROR) << "null pointer dereferencing.";
    return RET_NULL_PTR;
  }
  MS_ASSERT(graph->allTensors.size() > bnNode->inputIndex.at(1));
  auto bnWeightTensorIdxes = bnNode->inputIndex;
  bnWeightTensorIdxes.erase(bnWeightTensorIdxes.begin());
  if (bnWeightTensorIdxes.size() == CAFFE_BATCHNORM_OP_WEIGHT_NUM) {
    bnWeightTensors.meanTensor = graph->allTensors.at(bnWeightTensorIdxes[CAFFE_BATCHNORM_MEAN_INDEX]).get();
    bnWeightTensors.varianceTensor = graph->allTensors.at(bnWeightTensorIdxes[CAFFE_BATCHNORM_VARIANCE_INDEX]).get();
  } else if (bnWeightTensorIdxes.size() == TF_BATCHNORM_OP_WEIGHT_NUM) {
    bnWeightTensors.scaleTensor = graph->allTensors.at(bnWeightTensorIdxes[TF_BATCHNORM_SCALE_INDEX]).get();
    bnWeightTensors.biasTensor = graph->allTensors.at(bnWeightTensorIdxes[TF_BATCHNORM_BIAS_INDEX]).get();
    bnWeightTensors.meanTensor = graph->allTensors.at(bnWeightTensorIdxes[TF_BATCHNORM_MEAN_INDEX]).get();
    bnWeightTensors.varianceTensor = graph->allTensors.at(bnWeightTensorIdxes[TF_BATCHNORM_VARIANCE_INDEX]).get();
  } else {
    MS_LOG(ERROR) << "BatchNorm should has 2 or 4 weight tensors, current number of weight tensors: "
                  << bnWeightTensorIdxes.size();
    return RET_ERROR;
  }

  if (bnWeightTensors.meanTensor == nullptr) {
    MS_LOG(ERROR) << "BatchNorm's mean tensor is nullptr";
    return RET_ERROR;
  }

  if (bnWeightTensors.varianceTensor == nullptr) {
    MS_LOG(ERROR) << "BatchNorm's variance tensor is nullptr";
    return RET_ERROR;
  }
  bnChannel = bnWeightTensors.meanTensor->data.size() * sizeof(uint8_t) / sizeof(float);
  if (bnChannel <= 0) {
    MS_LOG(ERROR) << "BatchNorm's channel less or equal 0";
    return RET_ERROR;
  }
  bnMeanTensor = bnWeightTensors.meanTensor;
  if (bnChannel != bnWeightTensors.varianceTensor->data.size() * sizeof(uint8_t) / sizeof(float)) {
    MS_LOG(ERROR) << "conv kernel num expected to be equal to variance size";
    return RET_ERROR;
  }

  if (bnWeightTensors.scaleTensor != nullptr) {
    if (bnChannel != bnWeightTensors.scaleTensor->data.size() * sizeof(uint8_t) / sizeof(float)) {
      MS_LOG(ERROR) << "conv kernel num  expected to be equal to scale size";
      return RET_ERROR;
    }
  }

  if (bnWeightTensors.biasTensor != nullptr) {
    if (bnChannel != bnWeightTensors.biasTensor->data.size() * sizeof(uint8_t) / sizeof(float)) {
      MS_LOG(ERROR) << "conv kernel num expected to be equal to bias size";
      return RET_ERROR;
    }
  }
  return RET_OK;
}

STATUS BatchNormConvertScalePass::GetBnEpsilon(MetaGraphT *graph, float &eps) {
  if (graph == nullptr) {
    MS_LOG(ERROR) << "null pointer dereferencing.";
    return RET_NULL_PTR;
  }
  if (bnNode == nullptr) {
    MS_LOG(ERROR) << "null pointer dereferencing.";
    return RET_NULL_PTR;
  }
  if (bnNode->primitive->value.type == schema::PrimitiveType_FusedBatchNorm) {
    eps = bnNode->primitive->value.AsFusedBatchNorm()->epsilon;
  } else if (bnNode->primitive->value.type == schema::PrimitiveType_BatchNorm) {
    eps = bnNode->primitive->value.AsBatchNorm()->epsilon;
  } else {
    MS_LOG(ERROR) << "match pattern has error, not BatchNorm node";
    return RET_ERROR;
  }

  if (eps < EPS) {
    eps = EPS_DEFAULT_FLOAT;
  }
  return RET_OK;
}

BatchNormConvertScalePass::~BatchNormConvertScalePass() {
  if (this->transScale != nullptr) {
    delete (this->transScale);
  }
  if (this->transBias != nullptr) {
    delete (this->transBias);
  }
}
}  // namespace lite
}  // namespace mindspore



In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *conv_activation_fusion.h
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include "tools/optimizer/fusion/transform_batchnorm_to_scale.h"
#include <memory>
#include "src/param_value_lite.h"
#include "schema/inner/model_generated.h"
#include "src/ir/primitive_t_value.h"
#include "utils/utils.h"
#include "tools/optimizer/common/gllo_utils.h"
#include "securec/include/securec.h"

namespace mindspore::opt {
namespace {
const int32_t NCHW_DIM_C = 1;
constexpr size_t kCaffeBNMeanIndex = 2;
constexpr size_t kTFBNMeanIndex = 4;
bool IsBatchNode(const BaseRef &n) {
  if (utils::isa<CNodePtr>(n) || utils::isa<ValueNodePtr>(n)) {
    auto type = opt::GetCNodeType(n);
    return type == schema::PrimitiveType_BatchNorm || type == schema::PrimitiveType_FusedBatchNorm;
  }
  return false;
}
bool IsNotConvNode(const BaseRef &n) {
  if (utils::isa<CNodePtr>(n) || utils::isa<ValueNodePtr>(n)) {
    auto type = opt::GetCNodeType(n);
    return type != schema::PrimitiveType_Conv2D && type != schema::PrimitiveType_DepthwiseConv2D;
  }
  return true;
}
const AnfNodePtr MakeNewSaleNode() {
  auto attr = std::make_unique<schema::ScaleT>();
  attr->axis = NCHW_DIM_C;
  auto prim = std::make_unique<schema::PrimitiveT>();
  prim->value.value = attr.release();
  prim->value.type = schema::PrimitiveType_Scale;
  auto primTValue = std::make_shared<lite::PrimitiveTValue>(prim.release());
  auto value_node = NewValueNode(primTValue);
  return value_node;
}

}  // namespace
const BaseRef BatchNormToScaleFusion::DefinePattern() const {
  auto any_var = std::make_shared<CondVar>(IsNotConvNode);
  auto bn_var = std::make_shared<CondVar>(IsBatchNode);
  auto bn_mean_var = std::make_shared<CondVar>(IsParamNode);
  auto bn_variable_var = std::make_shared<CondVar>(IsParamNode);
  auto bn_other_var = std::make_shared<SeqVar>();
  return VectorRef({bn_var, any_var, bn_mean_var, bn_variable_var, bn_other_var});;
}
// BatchNorm weight Tensor definition:
// caffe
//   estimated_mean  --0
//   estimated_variance  --1
// tensorflow
//   scale    -- 0
//   bias        --1
//   estimated_mean  --2
//   estimated_variance  --3
const AnfNodePtr BatchNormToScaleFusion::Process(const FuncGraphPtr &func_graph, const AnfNodePtr &node,
                                                 const EquivPtr &) const {
  MS_ASSERT(nullptr != func_graph);
  MS_ASSERT(nullptr != node);
  auto bn_node = utils::cast<CNodePtr>(node);
  if (bn_node == nullptr) {
    MS_LOG(ERROR) << "batchnorm to scale fusion not found cnode.";
    return nullptr;
  }
  auto input_size = bn_node->inputs().size();
  if (input_size != 4 && input_size != 6) {
    return nullptr;
  }
  // tf or caffe mean tensor shape as bn channels
  AnfNodePtr bn_mean_node = nullptr;
  if (GetCNodeType(bn_node) == schema::PrimitiveType_BatchNorm) {
    bn_mean_node = bn_node->input(kCaffeBNMeanIndex);
  } else if (GetCNodeType(bn_node) == schema::PrimitiveType_FusedBatchNorm) {
    bn_mean_node = bn_node->input(kTFBNMeanIndex);
  } else {
    MS_LOG(EXCEPTION) << "not caffe or tf batchnorm op.";
  }
  auto bn_mean_param = bn_mean_node->cast<ParameterPtr>()->default_param();
  auto bn_mean_tensor = std::dynamic_pointer_cast<ParamValueLite>(bn_mean_param);
  auto bn_channels = bn_mean_tensor->tensor_shape_size();
  if (bn_channels <= 0) {
    MS_LOG(ERROR) << "Unsupported bn chanle size";
    return node;
  }
  auto trans_scale = new(std::nothrow) float[bn_channels];
  if (trans_scale == nullptr) {
    delete[] trans_scale;
    MS_LOG(ERROR) << "tensor_data is nullptr";
    return nullptr;
  }
  auto trans_bias = new(std::nothrow) float[bn_channels];
  if (trans_bias == nullptr) {
    MS_LOG(ERROR) << "tensor_data is nullptr";
    delete[] trans_bias;
    return nullptr;
  }
  GenTransParam(bn_node, bn_channels, trans_scale, trans_bias);
  auto scale_weight_node = AddNewBiasNode(trans_scale, func_graph, bn_channels, bn_mean_tensor);
  auto scale_bias_node = AddNewBiasNode(trans_bias, func_graph, bn_channels, bn_mean_tensor);
  std::vector<AnfNodePtr> op_inputs = {MakeNewSaleNode(), bn_node->input(1), scale_weight_node, scale_bias_node};
  auto new_scale_node = func_graph->NewCNode(op_inputs);
  new_scale_node->set_fullname_with_scope("scale_from_bn");
  new_scale_node->set_abstract(bn_node->abstract());
  return new_scale_node;
}
}  // namespace mindspore::opt


In [ ]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import torch


class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score


def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
def reduce_fn(vals):
    return sum(vals) / len(vals)

In [ ]:
!mkdir -p ./input/roberta-base

In [ ]:
save_path = './input/roberta-base'
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
config = RobertaConfig.from_pretrained('roberta-base')
tokenizer.save_vocabulary(save_path)
model.save_pretrained(save_path)
config.save_pretrained(save_path)

#config

In [ ]:
class config:
    LEARNING_RATE = 4e-5
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32
    EPOCHS = 3
    TRAINING_FILE = "./tweet-sentiment/train_8folds.csv"
    ROBERTA_PATH = "./input/roberta-base"
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
        vocab_file=f"{ROBERTA_PATH}/vocab.json", 
        merges_file=f"{ROBERTA_PATH}/merges.txt", 
        lowercase=True,
        add_prefix_space=True
    )

In [ ]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [ ]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

#TweetModel

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(config.ROBERTA_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [ ]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

#train

In [ ]:
def train_fn(data_loader, model, optimizer, device, num_batches, scheduler=None):
    model.train()
    tk0 = tqdm(data_loader, total=len(data_loader), desc="Training")
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
        scheduler.step()
        tk0.set_postfix(loss=loss.item())

In [ ]:
#calculate_jaccard_score

In [ ]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


def eval_fn(data_loader, model, device):
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader), desc="Validating")
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].cpu().numpy()

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=loss.item())

    return jaccards.avg

In [ ]:
dfx = pd.read_csv(config.TRAINING_FILE)

#run fold

In [ ]:
def run(fold):
    model_config = transformers.RobertaConfig.from_pretrained(config.ROBERTA_PATH)
    model_config.output_hidden_states = True
    MX = TweetModel(conf=model_config)
    
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)

    device = xm.xla_device(fold + 1)
    model = MX.to(device)

    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=1
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=1
    )

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = [
        "bias",
        "LayerNorm.bias",
        "LayerNorm.weight"
    ]
    optimizer_parameters = [
        {
            'params': [
                p for n, p in param_optimizer if not any(
                    nd in n for nd in no_decay
                )
            ], 
         'weight_decay': 0.001
        },
        {
            'params': [
                p for n, p in param_optimizer if any(
                    nd in n for nd in no_decay
                )
            ], 
            'weight_decay': 0.0
        },
    ]
    num_train_steps = int(
        len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS
    )
    optimizer = AdamW(
        optimizer_parameters, 
        lr=config.LEARNING_RATE
    )
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    best_jac = 0
    es = EarlyStopping(patience=2, mode="max")
    num_batches = int(len(df_train) / config.TRAIN_BATCH_SIZE)
    
    for epoch in range(config.EPOCHS):
        train_fn(
            train_data_loader, 
            model, 
            optimizer, 
            device,
            num_batches,
            scheduler
        )

        jac = eval_fn(
            valid_data_loader, 
            model, 
            device
        )
        print(f'Epoch={epoch}, Fold={fold}, Jaccard={jac}')
        if jac > best_jac:
            xm.save(model.state_dict(), f"model_{fold}.bin")
            best_jac = jac

In [ ]:
from joblib import Parallel,delayed

In [ ]:
Parallel(n_jobs=8, backend="threading")(delayed(run)(i) for i in range(8))